In [1]:
#!pip uninstall keras-nightly
#!pip uninstall -y tensorflow
#!pip uninstall -y protobuf
!pip install keras
!pip install tensorflow==2.2.0
!pip install h5py==2.10.0
#!pip install protobuf

!pip install --upgrade "protobuf<=3.20.1"

  Using cached protobuf-3.20.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.0
    Uninstalling protobuf-3.19.0:
      Successfully uninstalled protobuf-3.19.0


In [4]:
# for loading/processing the images

from tensorflow import keras
from tensorflow.python.keras.preprocessing.image import load_img 
from tensorflow.python.keras.preprocessing.image import img_to_array 
from tensorflow.python.keras.applications.vgg16 import preprocess_input 

# models 
from tensorflow.python.keras.applications.vgg16 import VGG16 
from tensorflow.python.keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cargar Fotos

In [5]:
path = r"/content/drive/MyDrive/DataSetML/train"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
cartas = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the cartas list
            cartas.append(file.name)

print(cartas)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/DataSetML/train'

In [5]:
print(len(cartas))

7989


Imagen to Array Numpy

In [6]:
# load the image as a 224x224 array
img = load_img(cartas[0], target_size=(224,224))
# convert from 'PIL.Image.Image' to numpy array
img = np.array(img)

print(img.shape)
#(224, 224, 3)

(224, 224, 3)


Añade dimensión

In [7]:
reshaped_img = img.reshape(1,224,224,3)
print(reshaped_img.shape)
#(1, 224, 224, 3)

x = preprocess_input(reshaped_img)

(1, 224, 224, 3)


Cargar vector para clustear imagenes

In [8]:
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

553467904/553467096 [==============================] - 25s 0us/step


In [9]:
features = model.predict(reshaped_img)
print(features.shape)

(1, 4096)


In [10]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

Ahora podemos usar esta función feature_extraction para extraer las características de todas las imágenes y almacenar las características en un diccionario con el nombre de archivo como clave.

In [ ]:
data = {}
p = r"/content/drive/MyDrive/DataSetML/train"

# lop through each image in the dataset
i=0

for carta in cartas:
    # try to extract the features and update the dictionary
    try:
        print(i)
        card = extract_features(carta,model)
        data[carta] = card
        i+=1
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)
          
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
feat.shape
#(210, 1, 4096)

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)
feat.shape
#(210, 4096)

# get the unique labels (from the flower_labels.csv)
df = pd.read_csv('flower_labels.csv')
label = df['label'].tolist()
unique_labels = list(set(label))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
pca = PCA(n_components=100, random_state=22)
pca.fit(card)
x = pca.transform(card)

print(f"Components before PCA: {f.shape[1]}")
print(f"Components after PCA: {pca.n_components}")

Hacer grupos de los clusters

In [ ]:
kmeans = KMeans(n_clusters=5,n_jobs=-1, random_state=22)
kmeans.fit(x)


# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)